In [79]:
# !pip install pytrends
# !npm install -g electron@6.1.4 orca

In [80]:
# # Colab Only:
# from google.colab import auth
# from google.colab import drive
# # drive.flush_and_unmount()
# # auth.authenticate_user()
# drive.mount('/gdrive')

In [81]:
import os 
import sys
import numpy as np
import pytrends
import time

In [82]:
ls /gdrive/My\ Drive/Colab\ Notebooks/COVID19/data/sucide

ls: /gdrive/My Drive/Colab Notebooks/COVID19/data/sucide: No such file or directory


In [83]:
import pandas as pd                        
from pytrends.request import TrendReq
pytrend = TrendReq()

In [84]:
ON_COLAB = True if os.path.isdir('/gdrive/My Drive/') else False
ROOT_DIR = '/gdrive/My Drive' if ON_COLAB else '/Users/magicii/Google Drive/'
DATA_DIR = '%s/Colab Notebooks/COVID19/preprocessed' % ROOT_DIR

DATA_GROUPS = {
    'job': ['employment', 'unemployment'],
    'anxiety': ['agitated', 'dread', 'overwhelming', 'sweat', 'anxiety', 'fear',
                'fatigue', 'panic', 'sweaty', 'anxious', 'headache', 'phobia', 
                'worry', 'back pain', 'insomnia', 'restless', 'worst', 'death', 
                'lightheadedness', 'sensitive', 'distress', 'muscle tension', 
                'sleep disorder', 'dizziness', 'nervous', 'stomach pain', 'anxious'],
    'disorder': ['anxiety', 'disorder', 'phobia', 'ptsd', 'anxiety disorder', 
                 'panic disorder', 'phobias', 'sleeping disorder'],
    'medication': ['antidepresant', 'klonopin', 'prozac', 'ssris', 'xanax', 'benzo', 
                   'lexapro', 'ssri', 'valium', 'zoloft'],
    'suicide': ['alcohol', 'drug', 'gun', 'reckless', 'unbearable', 
               'death', 'drugs', 'hopeless', 'suicide prevention', 'suicide methods'],
    'crypto': [],
    'memory': [],
    'opioid': ['opioid', 'drug', 'heroin', 'fentanyl', 'pain relievers', 'hydrocodone', 'codeine',
               'oxycodone', 'morphine']
}

FORCE_RELOAD_GROUPS = {
    'job': False,
    'anxiety': False,
    'disorder': False,
    'medication': False,
    'suicide': False,
    'crypto': False,
    'memory': False,
    'opioid': False,
}

DATA_GROUPS_MAIN_TERM = {
    'job': 'job',
    'anxiety': 'anxiety',
    'disorder': 'disorder',
    'medication': 'anxiety medication',
    'suicide': 'suicide',
    'crypto': 'crypto',
    'memory': 'memory',
    'opioid': 'opioid',
}

DATA_GROUPS_EXCLUDED_QUERIES = {
    'job': ['btc', 'bmw', 'benz', 'binance', 'apple', 'job lot'],
    'anxiety': ['dog anxiety'],
    'disorder': [],
    'medication': ['depression', 'depression and anxiety', 'anxiety disorder', 'dog anxiety'],
    'suicide': ['suicide squad', 'the suicide squad', 'suicide boys', 'suicide squad 2', 'joker', 
                'joker suicide squad', 'suicide lyrics', 'suicide squad cast', 
                'epstein suicide', 'harley quinn', 'suicide number', 'epstein', 
                'harley quinn suicide squad', 'suicidal'],
    'crypto': [],
    'memory': [],
    'opioid': [],
}

DATA_GROUPS_REPLACED_QUERIES = {
    'job': {},
    'anxiety': {
        'symptoms': 'anxiety symptoms',
    },
    'disorder': {
        'symptoms': 'disorder symptoms',
    },
    'medication': {},
    'suicide': [],
    'crypto': [],
    'memory': [],
    'opioid': [],
}


DATA_GROUPS_EXTENDED_QUERIES = {
    'job': [],
    'anxiety': [],
    'disorder': [],
    'medication': [],
    'suicide': [],
    'crypto': [],
    'memory': [],
    'opioid': [],
}

DATA_DIR

'/Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed'

In [85]:
FORCE_REDOWNLOAD_RELATED_TOPICS = False
FORCE_REDOWNLOAD_TRENDS = False

### Related queries

In [86]:
for idx, group in enumerate(DATA_GROUPS.keys()):
    main_term = DATA_GROUPS_MAIN_TERM[group]
    pytrend.build_payload(kw_list=[main_term], timeframe='today 12-m', geo = 'US')
    print('Group: %s' % group)
    
    # Related topics
    df_topics_top = None
    df_topics_rising = None
    if not os.path.isfile('%s/topics/%s_top.csv' % (DATA_DIR, group)) or FORCE_REDOWNLOAD_RELATED_TOPICS \
        or FORCE_RELOAD_GROUPS[group]:
        print('- Fetching related topics data...')
        related_topic = pytrend.related_topics()
        df_topics_top = related_topic[main_term]['top']
        df_topics_rising = related_topic[main_term]['rising']

        df_topics_top.to_csv('%s/topics/%s_top.csv' % (DATA_DIR, group), index=False)
        df_topics_rising.to_csv('%s/topics/%s_rising.csv' % (DATA_DIR, group), index=False)
        print('- Saved!')
    else:
        print('- Related topics files existed.')
        df_topics_top = pd.read_csv('%s/topics/%s_top.csv' % (DATA_DIR, group))
        df_topics_rising = pd.read_csv('%s/topics/%s_rising.csv' % (DATA_DIR, group))

    # Related queries
    df_queries_top = None
    df_queries_rising = None
    if not os.path.isfile('%s/queries/%s_top.csv' % (DATA_DIR, group)) or FORCE_REDOWNLOAD_RELATED_TOPICS:
        print('- Fetching related queries data...')
        related_querie = pytrend.related_queries()
        df_queries_top = related_querie[main_term]['top']
        df_queries_rising = related_querie[main_term]['rising']

        df_queries_top.to_csv('%s/queries/%s_top.csv' % (DATA_DIR, group), index=False)
        df_queries_rising.to_csv('%s/queries/%s_rising.csv' % (DATA_DIR, group), index=False)
        print('- Saved!')
    else:
        print('- Related queries files existed.')
        df_queries_top = pd.read_csv('%s/queries/%s_top.csv' % (DATA_DIR, group))
        df_queries_rising = pd.read_csv('%s/queries/%s_rising.csv' % (DATA_DIR, group))

    # Select queries
    for q in df_queries_top['query']:
        if q in DATA_GROUPS_EXCLUDED_QUERIES[group]:
            continue
        if q in DATA_GROUPS_REPLACED_QUERIES[group]:
            q = DATA_GROUPS_REPLACED_QUERIES[group][q]
        DATA_GROUPS_EXTENDED_QUERIES[group] += [q]
        
    # Reduce downloading freq
    time.sleep(2)

DATA_GROUPS_EXTENDED_QUERIES

Group: job
- Related topics files existed.
- Related queries files existed.
Group: anxiety
- Related topics files existed.
- Related queries files existed.
Group: disorder
- Related topics files existed.
- Related queries files existed.
Group: medication
- Related topics files existed.
- Related queries files existed.
Group: suicide
- Related topics files existed.
- Related queries files existed.
Group: crypto
- Related topics files existed.
- Related queries files existed.
Group: memory
- Related topics files existed.
- Related queries files existed.
Group: opioid
- Related topics files existed.
- Related queries files existed.


{'job': ['loans',
  'job hunting',
  'jobs',
  'job description',
  'get a job',
  'job openings',
  'job near me',
  'job application',
  'good job',
  'job search',
  'how to get a job',
  'job indeed',
  'jobs near me',
  'job hiring',
  'nose job',
  'job center',
  'job fair',
  'job service',
  'job resume'],
 'anxiety': ['anxiety depression',
  'depression',
  'anxiety disorder',
  'anxiety attack',
  'anxiety symptoms',
  'anxiety help',
  'what is anxiety',
  'depression and anxiety',
  'anxiety medication',
  'social anxiety',
  'help with anxiety',
  'symptoms of anxiety',
  'separation anxiety',
  'cbd',
  'cbd anxiety',
  'anxiety attacks',
  'sleep anxiety',
  'anxiety test',
  'high anxiety',
  'anxiety meds',
  'stress and anxiety',
  'cbd for anxiety',
  'anti anxiety',
  'health anxiety'],
 'disorder': ['personality disorder',
  'bipolar',
  'bipolar disorder',
  'eating disorder',
  'anxiety disorder',
  'anxiety',
  'borderline personality disorder',
  'borderline d

### Related topics

In [87]:
for group in DATA_GROUPS:
    group_queries = sorted(list(np.unique(DATA_GROUPS[group] + DATA_GROUPS_EXTENDED_QUERIES[group])))
    for query in group_queries:
        print('Query: %s' % query)
        query_file_name = query.lower().replace(' ', '_')
        query_file_path = '%s/trends/%s/%s.csv' % (DATA_DIR, group, query_file_name)
        if os.path.isfile(query_file_path) and not FORCE_REDOWNLOAD_TRENDS and not FORCE_RELOAD_GROUPS[group]:
            print('- File existed at %s' % query_file_path)
        else:
            pytrend.build_payload(kw_list=[query], timeframe='today 12-m', geo = 'US', )
            df = pytrend.interest_over_time()
            df['date'] = df.index
            df.to_csv(query_file_path, index=False)
            print('- Saved: %s' % query_file_path)
    # Reduce downloading freq
    time.sleep(2)

Query: employment
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/job/employment.csv
Query: get a job
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/job/get_a_job.csv
Query: good job
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/job/good_job.csv
Query: how to get a job
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/job/how_to_get_a_job.csv
Query: job application
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/job/job_application.csv
Query: job center
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/job/job_center.csv
Query: job description
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/job/job_description.csv
Query: job fair
- File existed at /Users/magicii/Google Drive//Colab Notebooks

Query: anxiety
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/disorder/anxiety.csv
Query: anxiety disorder
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/disorder/anxiety_disorder.csv
Query: autism
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/disorder/autism.csv
Query: bipolar
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/disorder/bipolar.csv
Query: bipolar disorder
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/disorder/bipolar_disorder.csv
Query: borderline disorder
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/disorder/borderline_disorder.csv
Query: borderline personality disorder
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/disorder/borderline_personality_disorder.csv
Query

Query: alcohol
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/suicide/alcohol.csv
Query: assisted suicide
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/suicide/assisted_suicide.csv
Query: coronavirus suicide
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/suicide/coronavirus_suicide.csv
Query: death
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/suicide/death.csv
Query: depression
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/suicide/depression.csv
Query: drug
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/suicide/drug.csv
Query: drugs
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/suicide/drugs.csv
Query: gun
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/pr

Query: codeine
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/opioid/codeine.csv
Query: drug
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/opioid/drug.csv
Query: drugs
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/opioid/drugs.csv
Query: epidemic
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/opioid/epidemic.csv
Query: fentanyl
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/opioid/fentanyl.csv
Query: heroin
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/opioid/heroin.csv
Query: hydrocodone
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/opioid/hydrocodone.csv
Query: morphine
- File existed at /Users/magicii/Google Drive//Colab Notebooks/COVID19/preprocessed/trends/opioid/morphine.cs

In [88]:
# df.reset_index().plot(x="geoName", y="coronavirus", figsize=(120, 10), kind ="bar")

### Plot

In [89]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
LINE_COLOR = 'rgb(130, 130, 130)'
LINE_COLOR_BEFORE = '#2375B3'
LINE_COLOR_AFTER = '#9D433C'

COVID_START_DATE = '2020-01-11'
COVID_START_DATE_X = '2020-01-12'
DATA_START_DATE = '2019-09-11'
DATA_END_DATE = '2020-07-20'

CAPTION = 'This is the caption'
IMAGE_DIR = '%s/Colab Notebooks/COVID19/figures' % ROOT_DIR

In [90]:
def plot_trends(group):
    group_queries = sorted(list(np.unique(DATA_GROUPS[group] + DATA_GROUPS_EXTENDED_QUERIES[group])))

    n_queries = len(group_queries)
    n_cols = 5
    n_rows = int(n_queries / n_cols) + (1 if n_queries % n_cols else 0)

    # Initialize figure with subplots
    subplot_titles = ['%s...' % t[:22] if len(t) >= 22 else t for t in group_queries]
    fig = make_subplots(
        rows=n_rows, cols=n_cols, subplot_titles=subplot_titles,
        shared_yaxes=True
    )

    for idx, query in enumerate(group_queries):
        row = int(idx / n_cols) + 1
        col = idx % n_cols + 1

        query_file_name = query.lower().replace(' ', '_')
        query_file_path = '%s/trends/%s/%s.csv' % (DATA_DIR, group, query_file_name)
        df = pd.read_csv(query_file_path)

        # Process
        df = df[(df['date'] >= DATA_START_DATE) & (df['date'] <= DATA_END_DATE)]
        df_before = df[(df['date'] <= COVID_START_DATE_X)]
        df_after = df[(df['date'] >= COVID_START_DATE)]

        # Normalize
        max_value = df[query].max()
        baseline = df_before[query].median()
        df['value'] = df[query].apply(lambda x: (x - baseline) / max_value)
        df_before['value'] = df_before[query].apply(lambda x: (x - baseline) / max_value)
        df_after['value'] = df_after[query].apply(lambda x: (x - baseline) / max_value)

        # Horizontal line 
        shape = go.layout.Shape(**{'type': 'line','y0':0,'y1': 0,'x0':str(df['date'].values[0]), 
                    'x1':str(df['date'].values[-1]),'xref':'x1','yref':'y1',
                    'line': {'color': 'rgb(200, 200, 200)','width': 1.5}})
        shape_Mar15 = go.layout.Shape(**{'type': 'line','y0':-0.25,'y1': 0.25,'x0':'2020-03-15', 
                    'x1':'2020-03-15','xref':'x1','yref':'y1',
                    'line': {'color': 'blue','width': 1.5, 'dash': 'dot'}})
        fig.add_shape(shape, row=row, col=col)
        fig.add_shape(shape_Mar15, row=row, col=col)

        # Plot
        subplot_before = go.Scatter(x=df_before['date'], y=df_before['value'], 
                            mode='lines',
                            line=dict(width=1, color=LINE_COLOR_BEFORE), 
                            line_shape='linear') # linear or spline 
        subplot_after = go.Scatter(x=df_after['date'], y=df_after['value'], 
                            mode='lines',
                            line=dict(width=1.5, color=LINE_COLOR_AFTER), 
                            line_shape='linear') # linear or spline 
        fig.add_trace(subplot_before, row=row, col=col)
        fig.add_trace(subplot_after, row=row, col=col)

    # Caption
    caption = go.layout.Annotation(
        showarrow=False,
        text=CAPTION,
        xanchor='center',
        x=0.5,
        yanchor='top',
        y=0.0,
        yshift=0,
    )

    # Layout
    fig.update_layout(title={'text': "Trends - Group: %s\n (%s - %s)" % (group, DATA_START_DATE, DATA_END_DATE), 'x':0.5, 'xanchor': 'center'}, 
                    height=n_rows * 175, width=250 * n_cols, coloraxis=dict(colorscale='Bluered_r'), 
                    showlegend=False, plot_bgcolor='rgb(255,255,255)', titlefont={'size': 30},
                    margin={'t': 200}
                    # annotations=[caption]
                    )
    fig.update_xaxes(showgrid=False, showticklabels=False, showline=False)
    fig.update_yaxes(showgrid=False, showticklabels=False, showline=False)

    # Save
    if not ON_COLAB:
        fig.write_image('%s/trends_group_%s.png' % (IMAGE_DIR, group))

    # Show
    fig.show()

In [91]:
plot_trends('anxiety')

/Users/magicii/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/magicii/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [92]:
plot_trends('disorder')


/Users/magicii/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/magicii/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [93]:
plot_trends('medication')

/Users/magicii/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/magicii/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [94]:
plot_trends('suicide')

/Users/magicii/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/magicii/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [95]:
plot_trends('job')

/Users/magicii/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/magicii/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [96]:
plot_trends('crypto')

/Users/magicii/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/magicii/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [97]:
plot_trends('memory')

/Users/magicii/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/magicii/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [98]:
plot_trends('opioid')


/Users/magicii/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/magicii/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

